In [2]:
import cv2
import json
import random
import logging
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import load_img
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

tf.get_logger().setLevel(logging.ERROR)

In [3]:
data = pd.read_csv('data.csv').drop(['Unnamed: 0'], axis=1).sort_values('ID', ascending=True)
data.head()

,CroppedHexBugCoordinationX,CroppedHexBugCoordinationY,OriginalBoxCoordinationX1,OriginalBoxCoordinationX2,OriginalBoxCoordinationY1,OriginalBoxCoordinationY2,Path,ID
497,197,95,923,1223,91,391,cropped_bugs/training04/frame0.jpg,0
396,41,31,591,891,218,518,cropped_bugs/training05/frame0.jpg,0
201,90,154,140,440,374,674,cropped_bugs/training01/frame0.jpg,0
100,197,95,908,1208,581,881,cropped_bugs/training03/frame0.jpg,0
302,104,175,66,366,263,563,cropped_bugs/training06/frame0.jpg,0


In [4]:
def compute_error(y, y_pred):
    error = np.sqrt(np.abs(y[0] - y_pred[0]) ** 2 + np.abs(y[1] - y_pred[1]) ** 2)
    return error

In [8]:
def compute_original_coordination(predicted_coordination, original_box_coordination):
    return (predicted_coordination[0] + original_box_coordination[0], predicted_coordination[1] + original_box_coordination[1])

In [5]:
reconstructed_model = tf.keras.models.load_model("./model_detail/v1.0/")

2023-04-25 13:15:42.054975: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-04-25 13:15:42.073618: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-04-25 13:15:42.073752: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [19]:
sample_id = 111
sample_img_path = data.iloc[sample_id].Path

original_image_path = f"../Videos/{sample_img_path.split('/')[1]}/{sample_img_path.split('/')[2]}"

print(original_image_path)

sample_img = np.array(load_img(sample_img_path))

expanded_img = np.expand_dims(sample_img, axis=0)
    
predicted = list(map(int, reconstructed_model.predict(expanded_img)[0]))
print(predicted)
radius = 5
color = (0, 255, 0)
thickness = 5

scaled_coordinations = compute_original_coordination(predicted, (data.iloc[sample_id].OriginalBoxCoordinationX1, data.iloc[sample_id].OriginalBoxCoordinationY1))

print(scaled_coordinations)

original_sample_img = np.array(load_img(original_image_path))

print(original_sample_img.shape)

annotated_img = cv2.circle(sample_img, predicted, radius, color, thickness)

original_annotated_img = cv2.circle(original_sample_img, scaled_coordinations, radius, color, thickness)

cv2.imshow('Sample', original_sample_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

../Videos/training03/frame22.jpg
1/1 [==============================] - 0s 12ms/step
[70, 35]
(420, 656)
(1080, 1210, 3)


In [5]:
sample_img_path = '../Videos/training04/frame19.jpg'

sample_img = np.array(load_img(sample_img_path))

expanded_img = np.expand_dims(sample_img, axis=0)
    
predicted = list(map(int, reconstructed_model.predict(expanded_img)[0]))
print(predicted)
radius = 5
color = (0, 255, 0)
thickness = 5

annotated_img = cv2.circle(sample_img, predicted, radius, color, thickness)

cv2.imshow('Sample', annotated_img)
cv2.waitKey(0)
cv2.destroyAllWindows()